In [1]:
!pip2 install BeautifulSoup4
!pip install lxml

In [9]:
from bs4 import BeautifulSoup
import requests

In [48]:
with open("SDA.tsv", "r") as inp:
    kegg_id = [ "hsa%05d"%int(x.strip().split()[0]) for x in inp.readlines()[1:]]
    
print(kegg_id)

for cur_id in kegg_id:
    link = "http://www.kegg.jp/kegg-bin/download?entry={}&format=kgml".format(cur_id)
    r = requests.get(link)
    with open("kegg_db/{}.kgml".format(cur_id), "w") as oup:
        oup.write(r.text)

['hsa04320', 'hsa04972', 'hsa04722', 'hsa05020', 'hsa00670', 'hsa00592', 'hsa00620', 'hsa03320', 'hsa04660', 'hsa04510', 'hsa03010', 'hsa05214', 'hsa04711', 'hsa00960']


In [19]:
with open("kegg_db/hsa00592.xml", "r") as inp:
    contents = inp.read()

In [20]:
bs = BeautifulSoup(contents)

In [21]:
from collections import defaultdict
gene = defaultdict(list)
for entry in bs.find_all('entry'):
    if entry['type'] == "gene":
        for hsa_id in entry["name"].split():
            link = "http://www.genome.jp/dbget-bin/www_bget?{}".format(hsa_id)
            print(link, )
            r = requests.get(link)
            print(r.status_code)
            cur_bs = BeautifulSoup(r.text, "html5lib")
            for a in cur_bs.find_all('a'):
                if a.has_key("href") and "genenames" in a["href"]:
                    gene[entry['id']].append(a.text)

('http://www.genome.jp/dbget-bin/www_bget?hsa:51',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:8310',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:51',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:8310',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:30',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:30',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:30',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:51',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:8310',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:100137049',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:11145',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:123745',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:151056',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:255189',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:26279',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:283748',)
200
('http://www.genome.jp/dbget-bin/www_bget?hsa:30814',)
200
('http://w

In [22]:
len(gene)

9

In [23]:
gene_int = dict()
for g in gene:
    gene_int[int(g)] = map(int, gene[g])

In [24]:
hgnc = dict()
with open("hgnc_to_symbol.tsv", "r") as inp:
    for line in inp.readlines()[1:]:
        hgnc_id, hgnc_sym = line.strip().split()
        hgnc_id = int(hgnc_id)
        hgnc[hgnc_id] = hgnc_sym

In [39]:
edges = []
for link in bs.find_all("relation"):
    e1 = int(link["entry1"])
    e2 = int(link["entry2"])
    type = str(link["type"])
    if e1 in gene_int and e2 in gene_int:
        for g1 in gene_int[e1]:
            for g2 in gene_int[e2]:
                edges.append("\t".join([hgnc[g1], hgnc[g2], type]))

In [40]:
nodes = set()
for n in gene_int:
    for x in gene_int[n]:
        nodes.add(hgnc[x])

In [41]:
edges = sorted(list(set(edges)))

In [42]:
len(edges)

24

In [31]:
hgnc = dict()
with open("mart_export.txt", "r") as inp:
    for line in inp.readlines()[1:]:
        #hgnc_id, hgnc_sym = line.strip().split()
        hgnc_sym, hgnc_id = line.strip().split()
        hgnc_id = int(hgnc_id)
        hgnc[hgnc_id] = hgnc_sym

In [43]:
with open("hsa05224_node.txt", "w") as oup:
    print >> oup, "\n".join(sorted(nodes))

In [46]:
with open("hsa05224_edge.txt", "w") as oup:
    print >> oup, "\n".join(sorted(edges))
    
edges

['ACOX1\tACAA1\tECrel',
 'ACOX3\tACAA1\tECrel',
 'FADS2\tPLA2G16\tECrel',
 'JMJD7-PLA2G4B\tFADS2\tECrel',
 'PLA2G10\tFADS2\tECrel',
 'PLA2G12A\tFADS2\tECrel',
 'PLA2G12B\tFADS2\tECrel',
 'PLA2G16\tFADS2\tECrel',
 'PLA2G1B\tFADS2\tECrel',
 'PLA2G2A\tFADS2\tECrel',
 'PLA2G2C\tFADS2\tECrel',
 'PLA2G2D\tFADS2\tECrel',
 'PLA2G2E\tFADS2\tECrel',
 'PLA2G2F\tFADS2\tECrel',
 'PLA2G3\tFADS2\tECrel',
 'PLA2G4A\tFADS2\tECrel',
 'PLA2G4B\tFADS2\tECrel',
 'PLA2G4C\tFADS2\tECrel',
 'PLA2G4D\tFADS2\tECrel',
 'PLA2G4E\tFADS2\tECrel',
 'PLA2G4F\tFADS2\tECrel',
 'PLA2G5\tFADS2\tECrel',
 'PLA2G6\tFADS2\tECrel',
 'PLB1\tFADS2\tECrel']

In [37]:
import glob
import fetch

In [47]:
reload(fetch)

for x in glob.glob("kegg_db/*.xml"):
    fetch.fetch(x)

fetching kegg_db/hsa00592.xml
fetching kegg_db/hsa00620.xml
fetching kegg_db/hsa00670.xml
fetching kegg_db/hsa00960.xml
fetching kegg_db/hsa03010.xml
fetching kegg_db/hsa03320.xml
fetching kegg_db/hsa04320.xml
fetching kegg_db/hsa04510.xml
fetching kegg_db/hsa04660.xml
fetching kegg_db/hsa04711.xml
fetching kegg_db/hsa04722.xml
fetching kegg_db/hsa04972.xml
fetching kegg_db/hsa05020.xml
fetching kegg_db/hsa05214.xml
